In [1]:
import pandas as pd
import re
df=pd.read_csv("/content/drive/MyDrive/Thesis Work 2021-2022/Latest dataset/BanglaNewsCommentData.csv")

In [2]:
df.head()

,Unnamed: 0,Comment,Tag
0,0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,0
1,1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,0
2,2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,0
3,3,চুরি নয় লুটপাট।,0
4,4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,2


In [3]:
df['Comment'] = df['Comment'].apply((lambda x: re.sub('[,?'']','',x)))

In [4]:
import numpy as np 
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding,SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Comment'].values)
X = tokenizer.texts_to_sequences(df['Comment'].values)
X = pad_sequences(X)

In [6]:
X

array([[   0,    0,    0, ...,   88,  302,  410],
       [   0,    0,    0, ...,   48,  961,  385],
       [   0,    0,    0, ...,  184,   50,  148],
       ...,
       [   0,    0,    0, ..., 1053,  165,  374],
       [   0,    0,    0, ...,   73,  287, 1526],
       [   0,    0,    0, ...,  406,  178,    8]], dtype=int32)

In [7]:
np.shape(X)

(13802, 90)

In [8]:
embedding_dimension = 128
input_length = 100
vocab_size = 5000
num_classes = 3
batch_size = 64
num_epochs = 5

In [12]:
import tensorflow as tf
from tensorflow import keras

In [13]:
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(vocab_size, embedding_dimension, input_length = input_length),
tf.keras.layers.Conv1D(128, 5, activation= 'relu'),
tf.keras.layers.MaxPooling1D(5),
tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True,dropout = 0.2)),
tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True,dropout = 0.2)),
tf.keras.layers.Dense(28, activation='relu'),
tf.keras.layers.Dense(14, activation='relu'),
keras.layers.Flatten(),
tf.keras.layers.Dense(num_classes , activation='softmax')])
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          640000    
                                                                 
 conv1d_1 (Conv1D)           (None, 96, 128)           82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 19, 128)          0         
 1D)                                                             
                                                                 
 bidirectional_2 (Bidirectio  (None, 19, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 19, 128)          98816     
 nal)                                                            
                                                        

In [14]:
Y = pd.get_dummies(df['Tag']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20,random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(11041, 90) (11041, 3)
(2761, 90) (2761, 3)


In [15]:
history=model.fit(X_train, Y_train, epochs = 10, batch_size=32, verbose = 2,validation_split=0.1)


Epoch 1/10


ValueError: ignored

In [ ]:
score = model.evaluate(X_train, Y_train,
                       batch_size=64, verbose=2)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

173/173 - 4s - loss: 0.5445 - acc: 0.7912 - 4s/epoch - 23ms/step
Train loss: 0.544456958770752
Train accuracy: 0.791232705116272


In [ ]:
score = model.evaluate(X_test, Y_test,
                       batch_size=64, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

44/44 - 1s - loss: 1.3191 - acc: 0.4995 - 1s/epoch - 23ms/step
Test loss: 1.319128155708313
Test accuracy: 0.4994567334651947


In [ ]:
y_preds=model.predict(X_test)

87/87 [==============================] - 2s 24ms/step


In [ ]:
y_preds

array([[0.18688245, 0.5713605 , 0.24175698],
       [0.28374395, 0.4034243 , 0.31283173],
       [0.8338505 , 0.01078724, 0.15536223],
       ...,
       [0.7781021 , 0.1832564 , 0.03864157],
       [0.6328395 , 0.35768396, 0.00947649],
       [0.89606446, 0.01030488, 0.09363068]], dtype=float32)

In [ ]:
y_test=Y_test.flatten()

In [ ]:
y_test

array([1, 0, 0, ..., 1, 0, 0], dtype=uint8)

In [ ]:
y_pred = y_preds.flatten()

In [ ]:
y_pred 

array([0.18688245, 0.5713605 , 0.24175698, ..., 0.89606446, 0.01030488,
       0.09363068], dtype=float32)

In [ ]:
y_predicted = np.where(y_pred > 0.5, 1, 0)
y_predicted

array([0, 1, 0, ..., 1, 0, 0])

In [ ]:
import sklearn.metrics

In [ ]:
r = sklearn.metrics.confusion_matrix(y_test, y_predicted)
print(r)

[[4364 1158]
 [1526 1235]]


In [ ]:
r = sklearn.metrics.classification_report(y_test, y_predicted)
print(r)

              precision    recall  f1-score   support

           0       0.74      0.79      0.76      5522
           1       0.52      0.45      0.48      2761

    accuracy                           0.68      8283
   macro avg       0.63      0.62      0.62      8283
weighted avg       0.67      0.68      0.67      8283

